In [1]:
'''Train CIFAR10 with PyTorch.'''
from __future__ import print_function
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import CosineAnnealingLR

from oncecycle import *

from torch import nn, optim
#from torch_lr_finder import LRFinder

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import csv

import models

#from utils import progress_bar, mixup_data, mixup_criterion
from utils import mixup_data, mixup_criterion

import numpy
import random

model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

print(model_names)

print(torch.__version__)

['dpn26', 'dpn92', 'fixup_resnet110', 'fixup_resnet1202', 'fixup_resnet20', 'fixup_resnet32', 'fixup_resnet44', 'fixup_resnet56', 'nobn_rezero_resnet56', 'preact_resnet101', 'preact_resnet101_rezero', 'preact_resnet152', 'preact_resnet152_rezero', 'preact_resnet18', 'preact_resnet18_rezero', 'preact_resnet34', 'preact_resnet34_rezero', 'preact_resnet50', 'preact_resnet50_rezero', 'resnet110', 'resnet1202', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet_v110', 'resnet_v1202', 'resnet_v20', 'resnet_v32', 'resnet_v44', 'resnet_v56', 'rezero_dpn26', 'rezero_dpn92', 'rezero_preactresnet101', 'rezero_preactresnet152', 'rezero_preactresnet18', 'rezero_preactresnet34', 'rezero_preactresnet50', 'rezero_resnet110', 'rezero_resnet1202', 'rezero_resnet20', 'rezero_resnet32', 'rezero_resnet44', 'rezero_resnet56', 'rezero_resnet602']
1.4.0


In [3]:
parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
parser.add_argument('-a', '--arch', metavar='ARCH', default='fixup_resnet110', choices=model_names, help='model architecture: ' +
                        ' | '.join(model_names) + ' (default: fixup_resnet110)')
parser.add_argument('--resume', '-r', action='store_true', help='resume from checkpoint')
parser.add_argument('--sess', default='mixup_default', type=str, help='session id')
parser.add_argument('--seed', default=random.randint(0,10000), type=int, help='rng seed')
parser.add_argument('--decay', default=3e-4, type=float, help='weight decay (default=3e-4)')
parser.add_argument('--batchsize', default=128, type=int, help='batch size per GPU (default=128)')
parser.add_argument('--n_epoch', default=200, type=int, help='total number of epochs')
parser.add_argument('--base_lr', default=0.1, type=float, help='base learning rate (default=0.1)')
parser.add_argument('--lr_step1', default=100, type=int, help='first lr step')
parser.add_argument('--lr_step2', default=150, type=int, help='second lr step')
parser.add_argument('--progress_bar', default='True', type=str, help='display progress bar')

args_input = [
   '-a', 'preact_resnet18',
    '--sess', 'superc', 
    '--batchsize','512',
    '--progress_bar', 'False',
    '--base_lr', '1.2',
    '--n_epoch', '65',
    '--decay', '2e-4'
]

global arg
args = parser.parse_args(args_input); args
args.progress_bar = (args.progress_bar=='True')
if args.progress_bar:
    from utils import progress_bar
    
    

torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
numpy.random.seed(args.seed)
random.seed(args.seed)
torch.backends.cudnn.deterministic = True

use_cuda = torch.cuda.is_available()
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
batch_size = args.batchsize
lr = args.base_lr



# Data
print('==> Preparing data..')
normalize = transforms.Normalize(mean=[0.4914 , 0.48216, 0.44653], std=[0.24703, 0.24349, 0.26159])
scale_size = 40
size=32
padding = int((scale_size - size) / 2)
transform_train = transforms.Compose([
        transforms.RandomCrop(size, padding=4),
        transforms.ColorJitter(.25,.25,.25),
        transforms.RandomRotation(2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



# Model
net = models.__dict__[args.arch]()

result_folder = './results/'
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

logname = result_folder + args.arch + '_' + args.sess + '_' + str(args.seed) + '.csv'

if use_cuda:
    net.cuda()
    #net = torch.nn.DataParallel(net)
    print('Using', torch.cuda.device_count(), 'GPUs.')
    cudnn.benchmark = True
    print('Using CUDA..')
    
#Training
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(),lr = 0.001,weight_decay = args.decay)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#lr_finder = LRFinder(net, optimizer, criterion, device)
#lr_finder.range_test(trainloader, end_lr=1, num_iter=100)
#lr_finder.plot()

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Using 1 GPUs.
Using CUDA..


In [4]:
scheduler = OneCycleLR(optimizer, lr_range = (1/25.* args.base_lr,args.base_lr),
                       num_steps = args.n_epoch * len(trainloader), annihilation_frac =.22,
                       reduce_factor = 0.01)



# Training
def train(epoch):
    #print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        
        outputs = net(inputs)
        optimizer.zero_grad()
        loss = criterion( outputs,targets )
        loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (torch.sum(predicted == targets.data)).cpu()
        
        acc = (100.*float(correct)/float(total))
        if args.progress_bar:
            progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (train_loss/(batch_idx+1), acc, correct, total))

    return (train_loss/batch_idx, acc)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion( outputs,targets )

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()
            if args.progress_bar:
                progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                    % (test_loss/(batch_idx+1), 100.*float(correct)/float(total), correct, total))

        # Save checkpoint.
        acc = 100.*float(correct)/float(total)
        if acc > best_acc:
            best_acc = acc
            checkpoint(acc, epoch)

    return (test_loss/batch_idx, acc)

def checkpoint(acc, epoch):
    # Save checkpoint.
    #print('Saving..')
    state = {
        'net': net,
        'acc': acc,
        'epoch': epoch,
        'rng_state': torch.get_rng_state()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + args.arch + '_' + args.sess + '_' + str(args.seed) + '.ckpt')


if not os.path.exists(logname):
    with open(logname[:-4]+'_args.csv', 'w') as logfile:
        logwriter = csv.writer(logfile, delimiter=',')
        logwriter.writerow(['filename = '+logname,'arch = ' +args.arch, 'base_lr = ' + str(args.base_lr), 'batchsize = '+str(args.batchsize), 'decay = '+ str(args.decay),
                          'n_epoch = ' +str(args.n_epoch),  'seed = '+str(args.seed) , 'sess = '+args.sess])
    with open(logname, 'w') as logfile:
        logwriter = csv.writer(logfile, delimiter=',')
        logwriter.writerow(['epoch', 'lr', 'train loss', 'train acc', 'test loss', 'test acc'])


In [5]:
best = 0

for epoch in range(start_epoch, args.n_epoch):
    for param_group in optimizer.param_groups:
        lr = param_group['lr']
    epoch_time = time.time()
    train_loss, train_acc = train(epoch)
    test_loss, test_acc = test(epoch)
    epoch_time = time.time() - epoch_time
    if best<test_acc:
        best = test_acc
    print('|{:2d}: |{:2.0f}s |'
          'lr {:02.4f} | '
          'train loss {:1.3f} |train acc {:2.2f}%|'
          'test loss {:1.3f} |test acc {:2.2f}% (best: {:2.2f}%)'.format(
            epoch,epoch_time,  lr,
            train_loss, train_acc, test_loss, test_acc,best))
    with open(logname, 'a') as logfile:
        logwriter = csv.writer(logfile, delimiter=',')
        logwriter.writerow([epoch, lr, train_loss, train_acc, test_loss, test_acc])


/home/thomas/ai/Fixup/cifar/models/preact_resnet.py:153: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(self.linear(out))


| 0: |47s |lr 0.0480 | train loss 2.023 |train acc 29.19%|test loss 1.690 |test acc 41.75% (best: 41.75%)
| 1: |42s |lr 0.0934 | train loss 1.607 |train acc 41.51%|test loss 1.615 |test acc 44.85% (best: 44.85%)
| 2: |42s |lr 0.1389 | train loss 1.397 |train acc 49.83%|test loss 1.290 |test acc 55.32% (best: 55.32%)
| 3: |42s |lr 0.1843 | train loss 1.179 |train acc 58.27%|test loss 1.084 |test acc 62.85% (best: 62.85%)
| 4: |42s |lr 0.2298 | train loss 1.005 |train acc 65.17%|test loss 0.990 |test acc 66.08% (best: 66.08%)
| 5: |42s |lr 0.2752 | train loss 0.879 |train acc 69.60%|test loss 1.129 |test acc 63.84% (best: 66.08%)
| 6: |42s |lr 0.3207 | train loss 0.745 |train acc 74.60%|test loss 0.760 |test acc 74.35% (best: 74.35%)
| 7: |42s |lr 0.3661 | train loss 0.675 |train acc 76.64%|test loss 0.692 |test acc 76.51% (best: 76.51%)
| 8: |42s |lr 0.4116 | train loss 0.631 |train acc 78.49%|test loss 0.919 |test acc 71.36% (best: 76.51%)
| 9: |42s |lr 0.4570 | train loss 0.579 |train

Traceback (most recent call last):
  File "/home/thomas/anaconda3/envs/ffnn/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/thomas/anaconda3/envs/ffnn/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/thomas/anaconda3/envs/ffnn/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/thomas/anaconda3/envs/ffnn/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [ ]:
for batch_idx, (inputs, targets) in enumerate(testloader):
    if batch_idx>0:
        break
    if use_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    print(inputs.size())
    print(net(inputs).size())

In [ ]:
for name, param in net.named_parameters():
    print(name)
    print(param)

In [ ]:
net

In [ ]:
net = models.preact_resnet18()

In [ ]:
nett = models.preact_resnet18()

for name, param in nett.named_parameters():
    print(name)
    print(param)